# File Input and Output

This brief tutorial focuses on the different pyGSTi objects that can be converted to & from text files.  Currently, `Model`, `DataSet`, and `MultiDataSet` objects, as well as lists and dictionaries of `Circuit` objects, can be saved to and loaded from text files.  All text-based input and output is done via the `pygsti.io` sub-package.  When objects have `save` and `load` methods (as `DataSet` objects do), these save and load *binary* formats which are different from the text formats used by the `pygsti.io` routines.  

Objects may also be serialized using Python's standard `pickle` package, or into the [JSON](https://www.json.org) and [MessagePack](https://msgpack.org/index.html) formats using `pygsti.io.json` and `pygsti.io.msgpack` respectively (see below for more details).  Serialization using these formats has the advantage of preserving all of an object's details (the text formats do this for the most part, but not perfectly), but results in a less- or non-human-readable  result.

## Text formats
Below we give examples of saving and loading each type of object to/from text format.  Many of these examples appear in other tutorials, but we thought it seemed useful to collect them into one place.

In [ ]:
import pygsti


#Models ------------------------------------------------------------
model_txt = \
"""
# Example text file describing a model

# State prepared, specified as a state in the Pauli basis (I,X,Y,Z)
PREP: rho0
LiouvilleVec
1/sqrt(2) 0 0 1/sqrt(2)

POVM: Mdefault

# State measured as yes (zero) outcome, also specified as a state in the Pauli basis
EFFECT: 0
LiouvilleVec
1/sqrt(2) 0 0 1/sqrt(2)

EFFECT: 1
LiouvilleVec
1/sqrt(2) 0 0 -1/sqrt(2)

END POVM

GATE: Gi
LiouvilleMx
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1

GATE: Gx
LiouvilleMx
1 0 0 0
0 1 0 0
0 0 0 -1
0 0 1 0

GATE: Gy
LiouvilleMx
1 0 0 0
0 0 0 1
0 0 1 0
0 -1 0 0

BASIS: pp 4
"""
with open("../tutorial_files/TestModel.txt","w") as f:
    f.write(model_txt)
    
target_model = pygsti.io.load_model("../tutorial_files/TestModel.txt")
pygsti.io.write_model(target_model, "../tutorial_files/TestModel.txt")


#DataSets ------------------------------------------------------------
dataset_txt = \
"""## Columns = 0 count, count total
{} 0 100
Gx 10 90
GxGy 40 60
Gx^4 20 90
"""
with open("../tutorial_files/TestDataSet.txt","w") as f:
    f.write(dataset_txt)

ds = pygsti.io.load_dataset("../tutorial_files/TestDataSet.txt")
pygsti.io.write_dataset("../tutorial_files/TestDataSet.txt", ds)


#MultiDataSets ------------------------------------------------------------
multidataset_txt = \
"""## Columns = DS0 0 count, DS0 1 count, DS1 0 frequency, DS1 count total                                
{} 0 100 0 100                                                                                                      
Gx 10 90 0.1 100                                                                                                    
GxGy 40 60 0.4 100                                                                                                  
Gx^4 20 80 0.2 100                                                                                                  
"""

with open("../tutorial_files/TestMultiDataSet.txt","w") as f:
    f.write(multidataset_txt)
    
multiDS = pygsti.io.load_multidataset("../tutorial_files/TestMultiDataSet.txt", cache=True)
pygsti.io.write_multidataset("../tutorial_files/TestDataSet.txt", multiDS)


#DataSets w/timestamped data --------------------------------------------
# Note: left of equals sign is letter, right is spam label
tddataset_txt = \
"""## 0 = 0
## 1 = 1
{} 011001
Gx 111000111
Gy 11001100
"""
with open("../tutorial_files/TestTDDataset.txt","w") as f:
    f.write(tddataset_txt)
    
tdds_fromfile = pygsti.io.load_tddataset("../tutorial_files/TestTDDataset.txt")
#NOTE: currently there's no way to *write* a DataSet w/timestamped data to a text file yet.


#Circuits ------------------------------------------------------------
from pygsti.modelpacks import smq1Q_XY  
cList = pygsti.construction.create_lsgst_circuits(
    [('Gxpi2',0), ('Gypi2',0)], smq1Q_XY.prep_fiducials(), smq1Q_XY.meas_fiducials(),
    smq1Q_XY.germs(), [1,2,4,8])    
pygsti.io.write_circuit_list("../tutorial_files/TestCircuitList.txt",cList,"#Test Circuit List")
pygsti.io.write_empty_dataset("../tutorial_files/TestEmptyDataset.txt",cList) 
  #additionally creates columns of zeros where data should go...
cList2 = pygsti.io.load_circuit_list("../tutorial_files/TestCircuitList.txt")


## Serialization to JSON and MSGPACK formats
PyGSTi contains support for reading and writing most (if not all) of its objects from and to the JSON and MessagePack formats.  The modules `pygsti.io.json` and `pygsti.io.msgpack` mimic the more general Python `json` and `msgpack` packages (`json` is a standard package, `msgpack` is a separate package, and must be installed if you wish to use pyGSTi's MessagePack functionality).  These, in turn, mimic the load/dump interface of the standard `pickle` module, so it's very easy to serialize data using any of these formats.  Here's a brief summary of the mais advantages and disadvantages of each format:

- pickle
 - **Advantages**: a standard Python package; very easy to use; can serialize almost anything.
 - **Disadvantages**: incompatibility between python2 and python3 pickles; can be large on disk (inefficient); not web safe.
- json
 - **Advantages**: a standard Python package; web-safe character set.; *should* be the same on python2 or python3
 - **Disadvantages**: large on disk (inefficient)
- msgpack
 - **Advantages**: *should* be the same on python2 or python3; efficient binary format (small on disk)
 - **Disadvantages**: needs external `msgpack` package; binary non-web-safe format.
 
We now demonstrate now to use the `io.json` and `io.msgpack` modules.  Using `pickle` is essentially the same, as pyGSTi objects support being pickled too.

In [ ]:
import pygsti.io.json as json
import pygsti.io.msgpack as msgpack

#Models
json.dump(target_model, open("../tutorial_files/TestModel.json",'w'))
target_model_from_json = json.load(open("../tutorial_files/TestModel.json"))

msgpack.dump(target_model, open("../tutorial_files/TestModel.mp",'wb'))
target_model_from_msgpack = msgpack.load(open("../tutorial_files/TestModel.mp", 'rb'))

#DataSets
json.dump(ds, open("../tutorial_files/TestDataSet.json",'w'))
ds_from_json = json.load(open("../tutorial_files/TestDataSet.json"))

msgpack.dump(ds, open("../tutorial_files/TestDataSet.mp",'wb'))
ds_from_msgpack = msgpack.load(open("../tutorial_files/TestDataSet.mp",'rb'))

#MultiDataSets
json.dump(multiDS, open("../tutorial_files/TestMultiDataSet.json",'w'))
multiDS_from_json = json.load(open("../tutorial_files/TestMultiDataSet.json"))

msgpack.dump(multiDS, open("../tutorial_files/TestMultiDataSet.mp",'wb'))
multiDS_from_msgpack = msgpack.load(open("../tutorial_files/TestMultiDataSet.mp",'rb'))

# Timestamped-data DataSets
json.dump(tdds_fromfile, open("../tutorial_files/TestTDDataset.json",'w'))
tdds_from_json = json.load(open("../tutorial_files/TestTDDataset.json"))

msgpack.dump(tdds_fromfile, open("../tutorial_files/TestTDDataset.mp",'wb'))
tdds_from_msgpack = msgpack.load(open("../tutorial_files/TestTDDataset.mp",'rb'))

#Circuit Lists
json.dump(cList, open("../tutorial_files/TestCircuitList.json",'w'))
cList_from_json = json.load(open("../tutorial_files/TestCircuitList.json"))

msgpack.dump(cList, open("../tutorial_files/TestCircuitList.mp",'wb'))
cList_from_msgpack = msgpack.load(open("../tutorial_files/TestCircuitList.mp",'rb'))